# IBM Data Science Course
## Data Science Capstone - The Battle of Neighborhoods (Week 1)
Lucas Friedrich Meincke<br>

>**Task 2**<br>
Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.
This submission will eventually become your **Data** section in your final report. So I recommend that you push the report (having your Data section) to your Github repository and submit a link to it.

## 2. Data

In this section are presented the description of the data and how they were used to solve the problem proposed.
<br><br>

### 2.1 Data Sources

* **Biggest cities in Germany:** these data were scraped from https://en.wikipedia.org/wiki/List_of_cities_in_Germany_by_population
* **The precipitation rate of each city in Germany:** these data were scraped from the main Wikipedia page of each city, through a loop. For example, for the city of Berlin, the data were scraped from https://de.wikipedia.org/wiki/Berlin, for the city of Stuttgart from https://de.wikipedia.org/wiki/Stuttgart, and so on.
* **The concurrence of other coffee and ice cream shops in each city:** once having the latitude and longitude information for each city in Germany, it was possible to collect the concurrence information through Foursquare API https://foursquare.com/. It was used two types of calls: regular calls for venue names, ids, and category collection and the premium calls to collect the score (customers' feedback) of each venue.

Many adjusts had to be made in the data for them to be properly used for further analysis. All the data preparation process is shown in section 2.3.
<br><br>

### 2.2 Data Usage

The first step was to collect a list that contains the **biggest cities in Germany** and also their latitude and longitude information. Knowing the name of the biggest cities, it is possible, though a loop funciton, to scrap the weather information containing the **precipitation rate** of each city from Wikipedia. At this point, it is possible to filter only a few cities which have the lowest precipitation rate and thus narrow the options for Heinz to open his open-air coffee and ice cream shop.

Once the best cities are filtered based on the climate condition, the **venues available** in each of these cities could be collected (through a regular call) and also their **scores** (through a premium call).

At this point, the data containing the availability of the venue was split: one dataframe containing all the venues in each city to compare them to Stuttgart, and a second dataframe containing only the venues that would be Heinz's concurrence (cafés and ice cream shops) and also their scores. By using both dataframes it was possible to select the city which would be the best option for him to open his shop based on both: the customers' feedback on the concurrence, and by using the k-clustering algorithm to find the most similar cities to Stuttgart.
<br><br>

### 2.3 Data Collection and Preparation
The data collection and preparation can be seen below alongside comments for each portion of code.
<br><br>

### 2.3.1 Python Libraries
The following libaries were used for this analysis

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

### 2.3.2 Biggest Cities in Germany
The data from the biggest cities in Germany were scraped from https://en.wikipedia.org/wiki/List_of_cities_in_Germany_by_population and after some adjusts transformed into a dataframe called **df_cities**

In [3]:
link1 = requests.get('https://en.wikipedia.org/wiki/List_of_cities_in_Germany_by_population').text
soup1 = BeautifulSoup(link1, 'html.parser')

l = []

for td in soup1.find('tbody').find_all('td'):
    data = td.text.strip()
    data = data.replace(',', '')
    data = data.split(" / ", 1)
    data = data[-1]
    data = data.split(" / ", 1)
    data = data[0]
    data = data.replace('°N', ';')
    data = data.replace('°E', '')
    data = data.replace('\ufeff', '')
    data = data.replace('\xa0', '')
    l.append(data)

rows = int((len(l)/9)) # number of rows is the lengh of the list / number of columns (converted to int)
l = np.reshape(l, (rows, 9)) # reshaping the list into matrix

df_cities = pd.DataFrame(l, columns = ["Rank", "City", "State", "Population", "2011 Census", 
                                      "Change", "Land Area", "Pop. Dens.", "Location"])

df_cities[['Latitude','Longitude']] = df_cities.Location.str.split(";", expand=True,).astype('float').round(3)
df_cities[["City", "State"]] = df_cities[["City", "State"]].astype('string')
df_cities["Population"] = df_cities["Population"].astype('int')

df_cities = df_cities[["City", "State", "Population", "Latitude", "Longitude"]]
df_cities.replace({'Halle (Saale)': 'Saale', 'Munich (München)': 'München', 'Cologne (Köln)': 'Köln',
                  'Hanover (Hannover)': 'Hannover', 'Nuremberg (Nürnberg)': 'Nürnberg'}, inplace=True)
df_cities.head()

,City,State,Population,Latitude,Longitude
0,Berlin,Berlin,3520031,52.517,13.383
1,Hamburg,Hamburg,1787408,53.550,10.000
2,München,Bavaria,1450381,48.133,11.567
3,Köln,North Rhine-Westphalia,1060582,50.933,6.950
4,Frankfurt am Main,Hesse,732688,50.117,8.683


### 2.3.2 Weather Conditions in the Cities
Once having the name of the biggest cities in Germany it is possible, though a loop, to scrap from the wiki pages of each city their weather conditions. The output prints "error" when the wiki page for a specific city does not have the weather condition information and also the count of the errors occured (in this case 19). The data that returned without error, after some adjusts, were transformed into a dataframe called **df_clima**

In [4]:
list_cities = list(df_cities["City"])
l = []
count = 0

for x in list_cities:
    try:
        link2 = requests.get(f'https://de.wikipedia.org/wiki/{x}').text
        soup2 = BeautifulSoup(link2, 'html.parser')
        tables = soup2.find('table', style='border: 5px solid #E5E5E5; font-size:95%; background:#E5E5E5; margin-bottom:10px;')  
        for row in tables.find_all('tr'):
            for td in row.find_all('td'):
                data = td.text.strip()
                data = data.replace('Jan', '').replace('Feb', '').replace('Mär', '').replace('Apr', '')
                data = data.replace('Mai', '').replace('Jun', '').replace('Jul', '').replace('Aug', '')
                data = data.replace('Sep', '').replace('Okt', '').replace('Nov', '').replace('Dez', '')
                data = data.replace(',', '.')
                data = data.replace('−', '-')
                l.append(data)
                l = list(filter(None, l))
            if data=="":
                continue
            else: l.append(x)
        print(x)
    except:
        count = count+1
        print('error')
        
print(count)
rows = int((len(l)/16)) # number of rows is the lengh of the list / number of columns (converted to int)
l = np.reshape(l, (rows, 16)) # reshaping the list into matrix

Berlin
Hamburg
München
Köln
Frankfurt am Main
Stuttgart
Düsseldorf
Dortmund
Essen
Leipzig
Bremen
Dresden
Hannover
Nürnberg
Duisburg
Bochum
Wuppertal
Bielefeld
Bonn
Münster
Karlsruhe
Mannheim
Augsburg
Wiesbaden
Gelsenkirchen
Mönchengladbach
Braunschweig
Chemnitz
Kiel
Aachen
error
Magdeburg
Freiburg im Breisgau
Krefeld
Lübeck
Oberhausen
Erfurt
Mainz
Rostock
Kassel
error
Hamm
Saarbrücken
error
Potsdam
error
error
Leverkusen
Osnabrück
Solingen
Heidelberg
Herne
Neuss
Darmstadt
error
Regensburg
error
Würzburg
error
error
Offenbach am Main
Ulm
Heilbronn
Pforzheim
error
error
Trier
error
error
Bremerhaven
Koblenz
Bergisch Gladbach
error
error
error
error
Siegen
error
error
Kaiserslautern
19


In [5]:
df_clima = pd.DataFrame(l, columns=["Measure", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul",
                        "Aug", "Sep", "Oct", "Nov", "Dec", "Sum/Avg", "Total", "City"])

df_clima = df_clima.astype({'Measure': 'string', 'Jan': 'float64', 'Feb': 'float64', 'Mar': 'float64',
                           'Apr': 'float64', 'May': 'float64', 'Jun': 'float64', 'Jul': 'float64',
                           'Aug': 'float64', 'Sep': 'float64', 'Oct': 'float64', 'Nov': 'float64',
                           'Dec': 'float64', 'Sum/Avg': 'string', 'Total': 'float64', 'City': 'string'})

mask = df_clima['Measure'].isin(['Regentage (d)', 'Sonnenstunden (h/d)', 'Luftfeuchtigkeit (%)',
                          'Temperatur (°C)', 'Rekordmaximum (°C)', 'Rekordminimum (°C)'])
df_clima = df_clima[~mask].reset_index(drop=True)
df_clima["Measure"] = df_clima['Measure'].replace(["Max. Temperatur (°C)", "Min. Temperatur (°C)", "Niederschlag (mm)"],
                                      ["Max. Temp. (°C)", "Min. Temp. (°C)", "Precipitation (mm)"])
df_clima.head()

,Measure,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Sum/Avg,Total,City
0,Max. Temp. (°C),2.9,4.2,8.5,13.2,18.9,21.6,23.7,23.6,18.8,13.4,7.1,4.4,Ø,13.4,Berlin
1,Min. Temp. (°C),-1.9,-1.5,1.3,4.2,9.0,12.3,14.3,14.1,10.6,6.4,2.2,-0.4,Ø,5.9,Berlin
2,Precipitation (mm),42.3,33.3,40.5,37.1,53.8,68.7,55.5,58.2,45.1,37.3,43.6,55.3,Σ,570.7,Berlin
3,Max. Temp. (°C),3.5,4.4,8.0,12.3,17.5,19.9,22.1,22.2,17.9,13.0,7.5,4.6,Ø,12.8,Hamburg
4,Min. Temp. (°C),-1.4,-1.2,1.1,3.3,7.4,10.5,12.7,12.5,9.6,6.0,2.4,0.0,Ø,5.3,Hamburg


In [6]:
df_clima.groupby('Measure')['City'].nunique()

Measure
Max. Temp. (°C)       59
Min. Temp. (°C)       59
Precipitation (mm)    60
Name: City, dtype: int64

### 2.3.3 The Main Dataframe
As only the precipitation rate and avarege temparatures during the spring, summer, and autumn (the seasons when the clients will be attended outside the shop) are needed, the dataframe containig the weather conditions was summarized by city and merged to **df_cities**. The dataframe **df_main** was then created containing the top 12 cities with less precipitation rate.

In [7]:
df_main = df_clima[['City', 'Measure']]
df_main['Total'] = (df_clima['Mar'] + df_clima['Apr'] + df_clima['May'] + df_clima['Jun'] + 
                         df_clima['Jul'] + df_clima['Aug'] + df_clima['Sep'] + df_clima['Oct'])

df_main = df_main.pivot(index='City', columns='Measure')['Total'].reset_index().round(3)
df_main["Summed Avg. Temp. (°C)"] = ((df_main["Max. Temp. (°C)"] + df_main["Min. Temp. (°C)"])/2/8).round(2)
df_main = pd.merge(df_main, df_cities, on ='City', how ='inner')
df_main = df_main[["City", "Precipitation (mm)", "Summed Avg. Temp. (°C)", "Population", "Latitude", "Longitude"]].round(2).sort_values(
                                by=["Precipitation (mm)", "Summed Avg. Temp. (°C)"], ascending=[True, False])
df_main = df_main.dropna().reset_index(drop=True)
df_main = df_main.head(12)
df_main

<ipython-input-7-5ab6e8513fc6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['Total'] = (df_clima['Mar'] + df_clima['Apr'] + df_clima['May'] + df_clima['Jun'] +


,City,Precipitation (mm),Summed Avg. Temp. (°C),Population,Latitude,Longitude
0,Magdeburg,347.6,12.96,235723,52.13,11.62
1,Leipzig,357.0,13.93,560472,51.33,12.38
2,Nürnberg,371.0,14.13,509975,49.45,11.08
3,Mainz,377.0,13.61,209779,50.00,8.27
4,Pforzheim,377.6,13.91,122247,48.90,8.72
5,Erfurt,384.0,12.03,210118,50.98,11.03
6,Dresden,387.0,14.74,543825,51.03,13.73
7,Berlin,396.2,13.37,3520031,52.52,13.38
8,Heidelberg,401.0,15.48,156267,49.42,8.72
9,Mannheim,401.0,15.21,305780,49.48,8.47


### 2.3.4 Exploring the Cities with Foursquare
The latitude and longitude informaton from **df_main** were used to explore the venues of each city generating the dataframe **df_venues_all** that will be used to comprate each city to Heinz's current home, Stuttgart. The Venues ID information was also collected so that it can be used further to collect the scores of each venue. Another data frame **df_venues_conc** was create containing only the venues that would be concurrence for Heinz's shop (Café, Koffee Shops, Ice Cream Shops and Bistros). In order to keep the data as cleaner as possible those 4 categories were recategorized into 2 (Café and Ice Cream Shop).

In [8]:
# credencials to be used on Foursquare API
CLIENT_ID = 'ZHZIJHIFEQCF4TUT3UOFXEVMSMTSOQ204ALBKXBCNMH2WXRA'
CLIENT_SECRET = '41I33OL4YE34QS5VWJXTNMSWCQFV143GKLFQ4OA1CQL0B2V4'
VERSION = '20180605'
LIMIT = 100 # A default Foursquare API limit value

In [9]:
# function to collect all the venues for each zip code
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City Name',
                             'Venue ID',
                             'Venue Name', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

# sending the parameters to the function
df_venues_all = getNearbyVenues(names=df_main['City'],
                            latitudes=df_main['Latitude'],
                            longitudes=df_main['Longitude'])

In [10]:
df_venues_conc = df_venues_all[df_venues_all['Venue Category'].str.contains('Ice|Café|Coffee Shop|Bistro')].reset_index(drop=True)
df_venues_conc.replace({'Coffee Shop': 'Café', 'Bistro': 'Café'}, inplace=True)
df_venues_conc.head()

,City Name,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Magdeburg,561a856e498ee6fc8b49cafb,Herzstück - Das Kuchenatelier,52.133728,11.615929,Café
1,Magdeburg,4cb5ddce1b0af04d8a64cc25,Il Capitello,52.126257,11.634307,Café
2,Magdeburg,4dc53e2345dd2645526a7f15,Eis-Konditorei Bortscheller,52.106756,11.640792,Ice Cream Shop
3,Leipzig,51e3cd828bbde0ebc2f0d32f,Handbrotzeit,51.341338,12.378071,Café
4,Leipzig,548137ca498eb80c2a45fc33,Espresso Zack Zack,51.332784,12.404315,Café


### 2.3.5 Collecting the Venues Scores with Foursquare
Finally, using the venues ID information, the customers' feedback (venue score) could be aquired and saved into the dataframe **df_ratings**.

In [15]:
def getVenuesScore(ID):
    
    score_list=[]
    for ids in ID:
        
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            ids,
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            LIMIT)
            
        results = requests.get(url).json()["response"]["venue"]
        score_list.append([results.get(v) for v in ["id", "rating", "ratingSignals"]])   

    scores = pd.DataFrame(score_list, columns = ["Venue ID", "Score", "Rating"])
    return(scores)

df_scores = getVenuesScore(ID=df_venues_conc['Venue ID'])
df_scores.head()

,Venue ID,Score,Rating
0,561a856e498ee6fc8b49cafb,8.0,9.0
1,4cb5ddce1b0af04d8a64cc25,7.5,14.0
2,4dc53e2345dd2645526a7f15,8.1,17.0
3,51e3cd828bbde0ebc2f0d32f,8.5,80.0
4,548137ca498eb80c2a45fc33,8.8,45.0


In [17]:
df_venues_conc = pd.merge(df_venues_conc, df_scores, on ='Venue ID', how ='outer')

,City Name,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Score,Rating
0,Magdeburg,561a856e498ee6fc8b49cafb,Herzstück - Das Kuchenatelier,52.133728,11.615929,Café,8.0,9.0
1,Magdeburg,4cb5ddce1b0af04d8a64cc25,Il Capitello,52.126257,11.634307,Café,7.5,14.0
2,Magdeburg,4dc53e2345dd2645526a7f15,Eis-Konditorei Bortscheller,52.106756,11.640792,Ice Cream Shop,8.1,17.0
3,Leipzig,51e3cd828bbde0ebc2f0d32f,Handbrotzeit,51.341338,12.378071,Café,8.5,80.0
4,Leipzig,548137ca498eb80c2a45fc33,Espresso Zack Zack,51.332784,12.404315,Café,8.8,45.0


### 2.5 Dataframes Summary
With the data collected so far is possible to help Heinz. A summary of the dataframes that will be used from now on is shown below:

**df_main**: dataframe containing the top 12 big cities with the lowest precipitaton rate (not counting the winter), their latitudes and longitude, but also their summed average temperature and population (these last 2 only for consult).<br>
**df_venues_all**: dataframe containing all the venues of the 12 cities so that they can be compared to Stuttgart, Heinz's hometown.<br>
**df_venues_conc**: dataframe containing the venues of the 12 cities that would be the concurrence for Heinz (Cafés and Ice Cream Shops) and their scores.<br>

In [18]:
df_main.head()

,City,Precipitation (mm),Summed Avg. Temp. (°C),Population,Latitude,Longitude
0,Magdeburg,347.6,12.96,235723,52.13,11.62
1,Leipzig,357.0,13.93,560472,51.33,12.38
2,Nürnberg,371.0,14.13,509975,49.45,11.08
3,Mainz,377.0,13.61,209779,50.00,8.27
4,Pforzheim,377.6,13.91,122247,48.90,8.72


In [19]:
df_venues_all.head()

,City Name,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Magdeburg,50520e06e4b0aa085324a1b1,Trattoria Cuochi per Caso,52.117225,11.627344,Trattoria/Osteria
1,Magdeburg,4bd464ebcfa7b713b4ee23da,Kulturhistorisches Museum,52.125470,11.629250,History Museum
2,Magdeburg,561a856e498ee6fc8b49cafb,Herzstück - Das Kuchenatelier,52.133728,11.615929,Café
3,Magdeburg,548db38e498eb00d713032f0,Hotel Motel One Magdeburg,52.126205,11.636149,Hotel
4,Magdeburg,4d9b602578f46ea8e36604fc,Orchidee,52.121743,11.630880,Vietnamese Restaurant


In [20]:
df_venues_conc.head()

,City Name,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Score,Rating
0,Magdeburg,561a856e498ee6fc8b49cafb,Herzstück - Das Kuchenatelier,52.133728,11.615929,Café,8.0,9.0
1,Magdeburg,4cb5ddce1b0af04d8a64cc25,Il Capitello,52.126257,11.634307,Café,7.5,14.0
2,Magdeburg,4dc53e2345dd2645526a7f15,Eis-Konditorei Bortscheller,52.106756,11.640792,Ice Cream Shop,8.1,17.0
3,Leipzig,51e3cd828bbde0ebc2f0d32f,Handbrotzeit,51.341338,12.378071,Café,8.5,80.0
4,Leipzig,548137ca498eb80c2a45fc33,Espresso Zack Zack,51.332784,12.404315,Café,8.8,45.0
